## 减小尺寸

In [4]:
import torch

import torch.nn as nn
import torch.nn.functional as F

# 大步长卷积 
conv_stride = nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=3,
                        stride=2, 
                        padding=1)

# 空洞卷积
conv_dilation = nn.Conv2d(in_channels=1,
                        out_channels=1,
                        kernel_size=3,
                        dilation=2,
                        padding=2,
                        padding_mode='zeros')
x = torch.rand(1, 1, 8, 8)

out_stride = conv_stride(x)
out_dilation = conv_dilation(x)

print(out_stride.shape) # torch.Size([1, 1, 4, 4]) 
print(out_dilation.shape) # torch.Size([1, 1, 8, 8])

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 8, 8])


In [6]:
torch.mean(x, dim=1, keepdim=True)

tensor([[[[5.8840e-02, 8.8304e-01, 3.0304e-02, 6.6571e-01, 3.2203e-01,
           9.7143e-02, 1.2706e-01, 3.5796e-01],
          [6.9173e-01, 3.7718e-01, 4.2464e-01, 4.2329e-01, 7.1095e-01,
           7.6326e-02, 7.4034e-01, 2.9528e-01],
          [2.0159e-01, 4.3785e-01, 9.1835e-01, 9.6365e-01, 1.2393e-01,
           1.7474e-01, 9.4890e-01, 1.1366e-01],
          [1.8123e-01, 6.4904e-01, 2.2894e-02, 6.6371e-01, 8.7870e-01,
           8.5400e-01, 6.1252e-01, 7.6460e-01],
          [3.7438e-01, 2.8792e-01, 9.7663e-01, 1.7397e-01, 8.3735e-01,
           8.2190e-01, 9.3596e-01, 9.0398e-01],
          [9.1881e-01, 4.1199e-04, 6.5208e-01, 6.5923e-01, 1.0790e-01,
           9.0119e-01, 8.0754e-01, 9.7555e-01],
          [9.4299e-02, 1.3071e-02, 3.7057e-02, 3.4672e-01, 5.8158e-01,
           9.4573e-01, 7.7415e-01, 9.9720e-01],
          [5.7956e-01, 1.0570e-01, 2.4595e-01, 7.6631e-01, 6.0523e-01,
           4.8122e-01, 5.0546e-01, 4.8314e-01]]]])

In [7]:
in_places=3
places=64
stride=1#不改变尺寸
expansion=4#通道倍增倍数
bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            # nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            # nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*expansion),
        )

In [8]:
x = torch.rand(1, 3, 256, 256)
bottleneck(x).shape

torch.Size([1, 256, 256, 256])

In [9]:
x=torch.rand(1, 3, 256, 256)
print(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=1, stride=1, padding=0, dilation=1)(x).shape)

torch.Size([1, 64, 256, 256])


## k、s、p、d

In [10]:
dilation=1
kernel_size=5
kernel_ = dilation*(kernel_size-1) + 1
stride=2
padding=2

conv1=nn.Conv2d(in_channels=3, out_channels=64, kernel_size=kernel_size, stride=stride,padding=padding,dilation=dilation)
conv1(x).shape


torch.Size([1, 64, 128, 128])

## 带有dilation的input和output

In [11]:
inputsize=256

kernel_size=3

stride=1
padding=0
dilation=0
kernel_ = dilation*(kernel_size-1) + 1
(inputsize+2*padding-kernel_)/stride+1

256.0

In [12]:
conv1_dila=nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2,padding=6,dilation=2)
conv1_dila(x).shape==conv1(x).shape

True

In [13]:
head=nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1))
conv_skip = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(64),
        )

In [14]:
class ResidualConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualConv, self).__init__()

        self.conv_block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(
                in_channels, out_channels, kernel_size=3, padding=1
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3,padding=1 ),
        )
        self.conv_skip = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
        )

    def forward(self, x):

        return self.conv_block(x) + self.conv_skip(x)
resc=ResidualConv(3,64)
resc(x).shape

torch.Size([1, 64, 256, 256])

In [15]:
head(x).shape

torch.Size([1, 64, 128, 128])

In [16]:
input_dim=3
output_dim=64
conv_block = nn.Sequential(
        nn.BatchNorm2d(input_dim),
        nn.ReLU(),
        nn.Conv2d(
            input_dim, output_dim, kernel_size=3, stride=1, padding=1
        ),
        nn.BatchNorm2d(output_dim),
        nn.ReLU(),
        nn.Conv2d(output_dim, output_dim, kernel_size=3, padding=1),
    )
conv_skip = nn.Sequential(
    nn.Conv2d(input_dim, output_dim, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(output_dim),
)
conv_block(x).shape

torch.Size([1, 64, 256, 256])

In [17]:
conv_skip(x).shape

torch.Size([1, 64, 256, 256])

In [19]:
input = torch.randn(3, 512, 64, 64)
# psa = PSA(channel=512, reduction=8)
# output = psa(input)
a = input.view(-1).sum()
a#.backward()

tensor(-633.3213)

In [ ]:
# 常用网络结构
from torchvision.models.segmentation import fcn_resnet101
model = fcn_resnet101(pretrained=False,num_classes=3)

d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
resnet34 = torchvision.models.resnet34(pretrained=False)
filters = [64, 128, 256, 512]

firstlayer = nn.Sequential(*list(resnet34.children())[:3])
maxpool = list(resnet34.children())[3]
encoder1 = resnet34.layer1
encoder1

d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, mome

In [ ]:
from torchvision.models.resnet import BasicBlock
BasicBlock(3,64)

BasicBlock(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)